In [116]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import time
import random
import requests
from bs4 import BeautifulSoup
import nltk
import cltk

nltk.download('stopwords')
# Rends nos tracés inline
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\melassouri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [117]:
# dfTrustPilote=pd.read_csv('trustpilot.csv')
# dfTrustPilote

df=pd.read_csv('corpus.csv')
df

,date,user,rating,review
0,Commenté en France le 9 août 2018,Justine,1.0,Le téléphone n a jamais fonctionné et Apple no...
1,Commenté en France le 21 juin 2018,Gouyer Eric,1.0,Soit-disant le téléphone est certifié recondit...
2,Commenté en France le 13 décembre 2018,Peineau A.,5.0,"Reçu aujourd'hui via UPS standard, bien emball..."
3,Commenté en France le 23 juillet 2018,Solenn Laurent,1.0,"iPhone reçu, la batterie ne tient pas une jour..."
4,Commenté en France le 1 janvier 2019,val,1.0,J’ai offert ce portable à mon fils pour Noël.....
...,...,...,...,...
1342,Commenté en France le 16 août 2020,Kasei,3.0,"au bout d'une semaine, le téléphone m'indique ..."
1343,Commenté en France le 2 juillet 2020,Josué KAZI,4.0,Produit fiable.J’ai reçu le téléphone depuis u...
1344,Commenté en France le 21 avril 2020,Familyismyreligion,5.0,"Livraison très rapide, plus rapide que prévu e..."
1345,Commenté en France le 21 septembre 2019,charret,3.0,la batterie se décharge très vite .


In [118]:
print("1: {}, 2: {}, 3: {}, 4: {}, 5: {}".format(
len(df[df['rating'] == 1.0]),
len(df[df['rating'] == 2.0]),
len(df[df['rating'] == 3.0]),
len(df[df['rating'] == 4.0]),
len(df[df['rating'] == 5.0])))



1: 348, 2: 93, 3: 129, 4: 235, 5: 542


In [119]:
import re
import string
import spacy
import fr_core_news_md
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from cltk.lemmatize.french.lemma import LemmaReplacer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score


nlp = fr_core_news_md.load()
        
def pre_processing(review):
    #tokenization
    words = re.split(r'\W+', review) 

#     print(words)

    #lowerCase
    words = [word.lower() for word in words] 

    #suppression de la ponctuation
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in words]

    # suppression des mots non alpha
    words = [word for word in stripped if word.isalpha()]

    #suppression des stop words
    stop_words = stopwords.words('french')
    words = [w for w in words if not w in stop_words]
    return words

In [120]:
def stemmatization(words):
    stemmer = FrenchStemmer()
    stemmed = [stemmer.stem(word) for word in words]  
#     print(words)
#     print(stemmed)
    return stemmed

In [121]:
def lemmatization(words):
    words_string = stringify(words)
    spacy_words = nlp(words_string)
    lemmed = [word.lemma_ for word in spacy_words]
#     print(lemmed)
    return lemmed

In [122]:
def stringify(words):
    str = ""
    for word in words:
        str += word+" "
    return str


In [123]:
def wordsCountsIDF(words):
    for word in words:
        if word in words_countsIDF:
            words_countsIDF[word] += 1
        else:
            words_countsIDF[word] = 1
def wordsCountsTF(words):
    for word in words:
        if word in words_countsTF:
            words_countsTF[word] += 1
        else:
            words_countsTF[word] = 1 
    for word in words:
            words_countsTF[word]/len(words) 

In [124]:
def vectorise(words):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(words)
    vector = vectorizer.transform(words)
    print(vector.shape)
    return vector

In [125]:
words_countsTF = {}     
words_countsIDF = {}    

TF = []
IDF = []
TFIDF = []

processed_review = []

def main():
    for review in df['review']: 
        words = pre_processing(review)
        words_stem = stemmatization(words)
        words_lem = lemmatization(words_stem)
        processed_review.append(stringify(words_lem))
main()
df['processed_review'] = processed_review 
x = vectorise(df['processed_review'])

df.loc[(df['rating'] > 3.0)]=1
df.loc[(df['rating'] < 3.0)]=0
y = df['rating']

(1347, 2161)


In [126]:
predictors = x # df["vectorised_review"] 
target = y # df["rating"]

#On decoupe le dataset en 2 (test_size=0.2) soit 20% de test et 80% de train qui vont respectivement dans les val 
#_test _train
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.2)

#On choisi un model pour predire notre resultat
cls=LogisticRegression(max_iter=200).fit(x_train,y_train)

#On calcul un score d'accurancy soit le nombre de prediction correcte obtenue
cls.score(x_val,y_val)

0.8925925925925926

In [127]:
cls=RandomForestClassifier(max_depth=12,n_estimators=300).fit(x_train,y_train)

cls.score(x_val,y_val)

0.8925925925925926

In [128]:
cls=GradientBoostingClassifier().fit(x_train,y_train)

cls.score(x_val,y_val)

0.8814814814814815